In [1]:
# %pip install --upgrade chromadb
# %pip install pillow
# %pip install open-clip-torch
# %pip install matplotlib
%pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [6]:
import sys
print(sys.executable)


/home/nafiz43/miniconda3/envs/fine_tune/bin/python


In [7]:
import PIL
print(PIL.__version__)


11.0.0


In [8]:
!pip show pillow


Name: pillow
Version: 11.0.0
Summary: Python Imaging Library (Fork)
Home-page: https://python-pillow.org
Author: 
Author-email: "Jeffrey A. Clark" <aclark@aclark.net>
License: MIT-CMU
Location: /home/nafiz43/miniconda3/envs/fine_tune/lib/python3.10/site-packages
Requires: 
Required-by: matplotlib, sentence-transformers


In [60]:
import chromadb
from chromadb.utils.embedding_functions import OpenCLIPEmbeddingFunction
from chromadb.utils.data_loaders import ImageLoader
from matplotlib import pyplot as plt
import os
import json
from glob import glob
from PIL import Image
import open_clip


# Create database file at folder "chroma" or load into client if exists.
chroma_client = chromadb.PersistentClient(path="chroma")

# Instantiate image loader helper.
image_loader = ImageLoader()

# Instantiate multimodal embedding function.
multimodal_ef = OpenCLIPEmbeddingFunction()

# Create the collection, aka vector database. Or, if database already exist, then use it. Specify the model that we want to use to do the embedding.
multimodal_db = chroma_client.get_or_create_collection(name="multimodal_db", embedding_function=multimodal_ef, data_loader=image_loader)



count = multimodal_db.count()  # Get the count of stored embeddings
print(f"Number of indexes (embeddings) in the collection: {count}")


# Assuming OpenCLIPEmbeddingFunction is already defined/imported
embedding_function = OpenCLIPEmbeddingFunction()  # Initialize your embedding function

# Define the directory paths
image_directory = '/mnt/data1/raiyan/breast_cancer/datasets/dmid/pixel_level_annotations/png_images/'
json_directory = '/mnt/data1/raiyan/breast_cancer/VLMs-for-Mammograms/GROUND-TRUTH-REPORTS'

# Initialize lists
ids = []
uris = []
metadatas = []
image_embeddings = []

# Initialize counter
processed_count = 0  

# Loop through images
for img_path in glob(os.path.join(image_directory, '*.png')):  
    img_name = os.path.basename(img_path)  
    json_path = os.path.join(json_directory, img_name.replace('.png', '.json'))  

    if os.path.exists(json_path):
        with open(json_path, 'r') as json_file:
            metadata = json.load(json_file)

        # Extract metadata fields
        metadata_entry = {
            'Breast_Composition': metadata.get('Breast_Composition', 'N/A'),
            'BIRADS': metadata.get('BIRADS', 'N/A'),
            'Findings': metadata.get('Findings', 'N/A')
        }

        # Load image
        image = Image.open(img_path).convert("RGB")

        image = preprocess(image).unsqueeze(0)  # Add batch dimension

        # Move to GPU if available
        device = "cuda" if torch.cuda.is_available() else "cpu"
        model = model.to(device)
        image = image.to(device)

        # Generate image embedding
        with torch.no_grad():
            image_embedding = model.encode_image(image)

        # Convert to NumPy array if needed
        image_embedding_np = image_embedding.cpu().numpy()
        image_embedding_list = image_embedding_np.flatten().tolist() 

        # print("Image embedding shape:", image_embedding_list.shape)


        # Store details
        ids.append(img_name)
        uris.append(img_path)
        metadatas.append(metadata_entry)
        # image_embeddings.append(image_embedding_np)  # Store embedding
        image_embeddings.append(image_embedding_list) 
        print(image_embeddings)

        # Increment counter and print progress
        processed_count += 1
        print(f"Indexed {processed_count}: {img_name}")

        if(processed_count==5):
            break

# Add records to the multimodal database

multimodal_db.add(
            ids=ids,
            uris=uris,
            embeddings=image_embeddings,  # Store OpenCLIP embeddings here
            metadatas=metadatas
        )
print(f"Total images indexed: {processed_count}")
count = multimodal_db.count()  # Get the count of stored embeddings
print(f"Number of indexes (embeddings) in the collection: {count}")

Number of indexes (embeddings) in the collection: 5
[[0.06715448200702667, -0.28943729400634766, 0.1586500108242035, -0.1720179319381714, -0.24536335468292236, -0.2740253210067749, -0.2404085397720337, 0.3617091178894043, 0.6524273753166199, -0.03629058599472046, 0.2061958760023117, 0.13514935970306396, 0.47249019145965576, 0.006526440382003784, 0.2468077391386032, -0.45620501041412354, 0.23792880773544312, 0.0338740199804306, -0.031583115458488464, 0.36638182401657104, -0.798420786857605, 0.02111729234457016, 0.5431272983551025, 0.6181768774986267, -0.13463051617145538, -0.44477754831314087, 0.280361533164978, -0.17557722330093384, -0.5133166313171387, -0.5125866532325745, 0.0782972052693367, 0.36224716901779175, 0.12255387008190155, -0.21244047582149506, 0.26790985465049744, -0.17674772441387177, 0.32657313346862793, -0.5952008366584778, -0.40960225462913513, -0.4361759126186371, -0.08663325756788254, -0.26619160175323486, -0.10411632806062698, -0.45701169967651367, 0.233909308910369

OperationalError: attempt to write a readonly database

In [62]:
import torch
from PIL import Image
import open_clip

# Load OpenCLIP model and preprocessing function
output = open_clip.create_model_and_transforms("ViT-B-32", pretrained="openai")

print(type(output), len(output))
model, preprocess, tokenizer = output  # Assuming it returns 3 values

tokenizer = open_clip.get_tokenizer("ViT-B-32")


# Load and preprocess the image
img_path = "/mnt/data1/raiyan/breast_cancer/datasets/dmid/pixel_level_annotations/png_images/IMG001.png"  # Change to your image path
image = Image.open(img_path).convert("RGB")
image = preprocess(image).unsqueeze(0)  # Add batch dimension

# Move to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
image = image.to(device)

# Generate image embedding
with torch.no_grad():
    image_embedding = model.encode_image(image)

# Convert to NumPy array if needed
image_embedding_np = image_embedding.cpu().numpy()

print("Image embedding shape:", image_embedding_np.shape)

print(image_embedding)


/home/nafiz43/miniconda3/envs/fine_tune/lib/python3.10/site-packages/open_clip/factory.py:388: UserWarning: These pretrained weights were trained with QuickGELU activation but the model config does not have that enabled. Consider using a model config with a "-quickgelu" suffix or enable with a flag.
  warnings.warn(


<class 'tuple'> 3
Image embedding shape: (1, 512)
tensor([[-2.2360e-01, -1.2817e-01,  2.3647e-01, -1.0251e-02, -2.4811e-01,
         -2.1990e-01, -4.7369e-01,  1.4421e-01,  8.4231e-01, -5.0378e-02,
          3.9330e-01, -9.0985e-02,  4.3056e-01, -4.5252e-02,  3.4723e-01,
         -5.4165e-01,  2.7178e-01, -2.9091e-01, -1.0851e-01,  3.0222e-01,
         -8.5886e-01, -4.1912e-02,  3.3400e-01,  5.1149e-01, -2.2953e-01,
         -2.5559e-01,  2.1656e-01, -2.3515e-01, -6.3728e-01, -5.5156e-01,
          1.1393e-01,  3.5448e-01, -5.5003e-02, -2.3737e-01,  8.7832e-02,
          2.8936e-01,  3.4693e-01, -2.7621e-01, -2.6740e-01, -2.6951e-01,
         -2.4104e-01, -3.9140e-01,  2.4331e-02, -4.6942e-01,  4.9617e-01,
         -5.5005e-02, -5.7648e-01,  3.2937e-01, -4.2877e-01, -2.6947e-01,
          3.8435e-01,  3.1902e-01, -5.9062e-02, -6.3537e-01,  3.3436e-03,
          3.9127e-01, -1.3117e-01, -9.9093e-02, -2.3860e-01,  8.9198e-02,
          5.0791e-01, -2.2850e-01,  3.5252e-02, -4.3257e-02, -

In [ ]:
import os
import json
from glob import glob
from PIL import Image
import open_clip

# Assuming OpenCLIPEmbeddingFunction is already defined/imported
embedding_function = OpenCLIPEmbeddingFunction()  # Initialize your embedding function

# Define the directory paths
image_directory = '/mnt/data1/raiyan/breast_cancer/datasets/dmid/pixel_level_annotations/png_images/'
json_directory = '/mnt/data1/raiyan/breast_cancer/VLMs-for-Mammograms/GROUND-TRUTH-REPORTS'

# Initialize lists
ids = []
uris = []
metadatas = []
image_embeddings = []

# Initialize counter
processed_count = 0  

# Loop through images
for img_path in glob(os.path.join(image_directory, '*.png')):  
    img_name = os.path.basename(img_path)  
    json_path = os.path.join(json_directory, img_name.replace('.png', '.json'))  

    if os.path.exists(json_path):
        with open(json_path, 'r') as json_file:
            metadata = json.load(json_file)

        # Extract metadata fields
        metadata_entry = {
            'Breast_Composition': metadata.get('Breast_Composition', 'N/A'),
            'BIRADS': metadata.get('BIRADS', 'N/A'),
            'Findings': metadata.get('Findings', 'N/A')
        }

        # Load image
        image = Image.open(img_path).convert("RGB")

        image = preprocess(image).unsqueeze(0)  # Add batch dimension

        # Move to GPU if available
        device = "cuda" if torch.cuda.is_available() else "cpu"
        model = model.to(device)
        image = image.to(device)

        # Generate image embedding
        with torch.no_grad():
            image_embedding = model.encode_image(image)

        # Convert to NumPy array if needed and flatten
        image_embedding_np = image_embedding.cpu().numpy()
        image_embedding_list = image_embedding_np.flatten().tolist()  # Convert to list

        # Store details
        ids.append(img_name)
        uris.append(img_path)
        metadatas.append(metadata_entry)
        image_embeddings.append(image_embedding_list)  # Store flattened embedding

        # Increment counter and print progress
        processed_count += 1
        print(f"Indexed {processed_count}: {img_name}")

        if processed_count == 5:
            break  # Stop after 5 images for testing

# Check data before adding to DB (Optional)
print("IDs:", ids)
print("URIs:", uris)
print("Embeddings:", image_embeddings)
print("Metadatas:", metadatas)

# Add records to the multimodal database

print(image_embeddings)
multimodal_db.add(
    ids=ids,
    uris=uris,
    embeddings=image_embeddings,  # Store OpenCLIP embeddings here
    metadatas=metadatas
)

# Final stats
print(f"Total images indexed: {processed_count}")
count = multimodal_db.count()  # Get the count of stored embeddings
print(f"Number of indexes (embeddings) in the collection: {count}")


Indexed 1: IMG315.png
Indexed 2: IMG156.png
Indexed 3: IMG282.png
Indexed 4: IMG062.png
Indexed 5: IMG145.png
IDs: ['IMG315.png', 'IMG156.png', 'IMG282.png', 'IMG062.png', 'IMG145.png']
URIs: ['/mnt/data1/raiyan/breast_cancer/datasets/dmid/pixel_level_annotations/png_images/IMG315.png', '/mnt/data1/raiyan/breast_cancer/datasets/dmid/pixel_level_annotations/png_images/IMG156.png', '/mnt/data1/raiyan/breast_cancer/datasets/dmid/pixel_level_annotations/png_images/IMG282.png', '/mnt/data1/raiyan/breast_cancer/datasets/dmid/pixel_level_annotations/png_images/IMG062.png', '/mnt/data1/raiyan/breast_cancer/datasets/dmid/pixel_level_annotations/png_images/IMG145.png']
Embeddings: [[0.036812663078308105, -0.2757193446159363, 0.27890631556510925, -0.11623577773571014, -0.205768421292305, -0.2145988941192627, -0.20198926329612732, 0.3184220790863037, 0.5997042059898376, -0.017445936799049377, 0.18894921243190765, 0.07917708903551102, 0.5191999077796936, 0.009415984153747559, 0.23826253414154053, -

OperationalError: attempt to write a readonly database

In [68]:
# Perform the search


img_path = "/mnt/data1/raiyan/breast_cancer/datasets/dmid/pixel_level_annotations/png_images/IMG001.png"  # Change to your image path
image = Image.open(img_path).convert("RGB")
image = preprocess(image).unsqueeze(0)  # Add batch dimension

# Move to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
image = image.to(device)

# Generate image embedding
with torch.no_grad():
    image_embedding = model.encode_image(image)

# Convert to NumPy array if needed
image_embedding_np = image_embedding.cpu().numpy()

# print("Image embedding shape:", image_embedding_np.shape)

# print(image_embedding)

print()

# print(image_embedding_np)




results = multimodal_db.query(
    query_embeddings=image_embedding_np,  # Pass the query embedding as a list
    n_results=2  # Number of similar results to return
)

# Print the results
# print("Search results:")
# for result in results:
#     print(f"ID: {result['id']}")
#     print(f"URI: {result['uri']}")
#     print(f"Metadata: {result['metadata']}")
#     print(f"Score: {result['score']}")  # Similarity score
#     print("-" * 50)
results

{'ids': [['IMG315.png', 'IMG062.png']],
 'embeddings': None,
 'documents': [[None, None]],
 'uris': None,
 'data': None,
 'metadatas': [[{'BIRADS': 4,
    'Breast_Composition': 'fibro fatty and glandular breast parenchyma (ACR-C).',
    'Findings': 'Well-defined lobulated soft opacity seen in the upper quadrant of the breast suggests high suspicious malignant lesion (BIRADS 4c)\nskin nipple and pectoral muscle appear normal.\nno abnormal microcalcification seen.\ntwo enlarged axillary nodes seen'},
   {'BIRADS': 1,
    'Breast_Composition': 'fibro glandular breast parenchyma (ACR B).',
    'Findings': 'few punctate discrete foci of calcifications seen. \nwell-defined oval soft opacity seen adjacent to the normally positioned nipple may suggest inverted nipple \nshadow or abnormal lesion. Ultrasound correlation is suggested. \nSkin appears normal. \nMultiple axillary adenopathy seen.'}]],
 'distances': [[12.863257110692482, 18.83533203744024]],
 'included': [<IncludeEnum.distances: 'dis

In [79]:
results["metadatas"][0]

[{'BIRADS': 4,
  'Breast_Composition': 'fibro fatty and glandular breast parenchyma (ACR-C).',
  'Findings': 'Well-defined lobulated soft opacity seen in the upper quadrant of the breast suggests high suspicious malignant lesion (BIRADS 4c)\nskin nipple and pectoral muscle appear normal.\nno abnormal microcalcification seen.\ntwo enlarged axillary nodes seen'},
 {'BIRADS': 1,
  'Breast_Composition': 'fibro glandular breast parenchyma (ACR B).',
  'Findings': 'few punctate discrete foci of calcifications seen. \nwell-defined oval soft opacity seen adjacent to the normally positioned nipple may suggest inverted nipple \nshadow or abnormal lesion. Ultrasound correlation is suggested. \nSkin appears normal. \nMultiple axillary adenopathy seen.'}]

In [88]:
results["ids"][0]

['IMG315.png', 'IMG062.png']

In [96]:
context = ""
for index in range(0, 2):
    image_id = results["ids"][0][index]  # Get the image ID
    img_findings = results["metadatas"][0][index]['Findings']  # Get the findings
    img_birads = results["metadatas"][0][index]['BIRADS']  # Get the BIRADS score
    img_breast_composition = results["metadatas"][0][index]['Breast_Composition']  # Get the breast composition

    # Dynamically build the context string with the actual values
    context += f"""
    Example :{index+1}
    {{
    "IMG-ID": /mnt/data1/raiyan/breast_cancer/datasets/dmid/pixel_level_annotations/png_images/{image_id},
    "BREAST-COMPOSITION": "{img_breast_composition}",
    "BIRADS": "{img_birads}",
    "FINDINGS": "{img_findings}"
    }}
    """

print(context)



    Example :1
    {
    "IMG-ID": /mnt/data1/raiyan/breast_cancer/datasets/dmid/pixel_level_annotations/png_images/IMG315.png,
    "BREAST-COMPOSITION": "fibro fatty and glandular breast parenchyma (ACR-C).",
    "BIRADS": "4",
    "FINDINGS": "Well-defined lobulated soft opacity seen in the upper quadrant of the breast suggests high suspicious malignant lesion (BIRADS 4c)
skin nipple and pectoral muscle appear normal.
no abnormal microcalcification seen.
two enlarged axillary nodes seen"
    }
    
    Example :2
    {
    "IMG-ID": /mnt/data1/raiyan/breast_cancer/datasets/dmid/pixel_level_annotations/png_images/IMG062.png,
    "BREAST-COMPOSITION": "fibro glandular breast parenchyma (ACR B).",
    "BIRADS": "1",
    "FINDINGS": "few punctate discrete foci of calcifications seen. 
well-defined oval soft opacity seen adjacent to the normally positioned nipple may suggest inverted nipple 
shadow or abnormal lesion. Ultrasound correlation is suggested. 
Skin appears normal. 
Multiple a

'fibro fatty and glandular breast parenchyma (ACR-C).'

'Well-defined lobulated soft opacity seen in the upper quadrant of the breast suggests high suspicious malignant lesion (BIRADS 4c)\nskin nipple and pectoral muscle appear normal.\nno abnormal microcalcification seen.\ntwo enlarged axillary nodes seen'

In [ ]:




img_IDs = results["ids"][0]
img_breast_compositions = results["ids"][0]
img_findings = 
img_birads = 

print(img_ID[0])

SyntaxError: invalid syntax (3508358150.py, line 1)

In [ ]:
len(image_embeddings[0])
image_embeddings

512

In [46]:
count = multimodal_db.count()  # Get the count of stored embeddings
print(f"Number of indexes (embeddings) in the collection: {count}")

Number of indexes (embeddings) in the collection: 5


In [50]:
# import os
# import json
# from glob import glob

# # Define the directory paths
# image_directory = '/mnt/data1/raiyan/breast_cancer/datasets/dmid/pixel_level_annotations/png_images/'  # Replace with the directory containing images
# json_directory = '/mnt/data1/raiyan/breast_cancer/VLMs-for-Mammograms/GROUND-TRUTH-REPORTS'    # Replace with the directory containing JSON files

# # Initialize empty lists for ids, uris, and metadatas
# ids = []
# uris = []
# metadatas = []

# # Initialize counters
# image_count = 0
# json_found_count = 0
# json_missing_count = 0

# # Loop through all image files in the directory
# for img_path in glob(os.path.join(image_directory, '*.png')):  # Adjust extension if needed
#     img_name = os.path.basename(img_path)  # Get the image file name (e.g., 'lion.jpg')
#     json_path = os.path.join(json_directory, img_name.replace('.png', '.json'))  # Construct the JSON file path
    
#     # Check if the corresponding JSON file exists
#     image_count += 1  # Increment image count for each processed image
    
#     if os.path.exists(json_path):
#         json_found_count += 1
#         # Read metadata from the JSON file
#         with open(json_path, 'r') as json_file:
#             metadata = json.load(json_file)
        
#         # Extracting specific metadata fields
#         # You can adjust the structure of this part to fit your JSON content
#         metadata_entry = {
#             'Breast_Composition': metadata.get('Breast_Composition', 'N/A'),
#             'BIRADS': metadata.get('BIRADS', 'N/A'),
#             'Findings': metadata.get('Findings', 'N/A')
#         }
        
#         # Add the details to the lists
#         ids.append(img_name)  # Use image file name as the ID
#         uris.append(img_path)  # Absolute path of the image
#         metadatas.append(metadata_entry)  # Add metadata as a dictionary (not a list)
#         print(f"Processed {img_name} (JSON found)")  # Print when JSON is found for an image
#         # Print img_name, img_path, and the metadata entry when JSON is found
#         print(f"Processed {img_name} (JSON found)")
#         print(f"Image Path: {img_path}")
#         print(f"Metadata: {metadata_entry}")
#         print("=" * 50)  # Just to separate the outputs for readability
#     else:
#         json_missing_count += 1
#         print(f"Processed {img_name} (JSON missing)")  # Print when JSON is missing for an image


# # Add the records to the multimodal database
# multimodal_db.add(
#     ids=ids,
#     uris=uris,
#     metadatas=metadatas
# )


In [49]:
# # Simple function to print the results of a query.
# # The 'results' is a dict {ids, distances, data, ...}
# # Each item in the dict is a 2d list.
# def print_query_results(query_list: list, query_results: dict)->None:
#     result_count = len(query_results['ids'][0])

#     for i in range(len(query_list)):
#         print(f'Results for query: {query_list[i]}')

#         for j in range(result_count):
#             id       = query_results["ids"][i][j]
#             distance = query_results['distances'][i][j]
#             data     = query_results['data'][i][j]
#             document = query_results['documents'][i][j]
#             metadata = query_results['metadatas'][i][j]
#             uri      = query_results['uris'][i][j]

#             print(f'id: {id}, distance: {distance}, metadata: {metadata}, uri: {uris}') 

#             # Display image, the physical file must exist at URI.
#             # (ImageLoader loads the image from file)
#             print(f'data: {uri}')
#             plt.imshow(data)
#             plt.axis("off")
#             plt.show()

{'ids': [['IMG315.png']],
 'embeddings': None,
 'documents': [[None]],
 'uris': None,
 'data': None,
 'metadatas': [[{'BIRADS': 4,
    'Breast_Composition': 'fibro fatty and glandular breast parenchyma (ACR-C).',
    'Findings': 'Well-defined lobulated soft opacity seen in the upper quadrant of the breast suggests high suspicious malignant lesion (BIRADS 4c)\nskin nipple and pectoral muscle appear normal.\nno abnormal microcalcification seen.\ntwo enlarged axillary nodes seen'}]],
 'distances': [[14.451730494864238]],
 'included': [<IncludeEnum.distances: 'distances'>,
  <IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [ ]:
# It is possible to submit multiple queries at the same time, just add to the list.  
query_texts = ['breast']

# Query vector db
query_results = multimodal_db.query(
    query_texts = query_texts,
    n_results=5,
    include=[  'metadatas', 'ids', 'uris'],
    where={'BIRADS':'3'}
)

print_query_results(query_texts, query_results)

ValueError: Expected include item to be one of documents, embeddings, metadatas, distances, uris, data, got ids in query.

In [ ]:
# Query the collection to retrieve some records and their metadata
query_results = multimodal_db.query(
    query_embeddings=None,  # You don't need embeddings for just retrieving metadata
    n_results=5,  # Specify the number of results to retrieve
    include=["ids", "metadatas", "documents", "uris"]  # Include metadata along with other fields
)

# Print metadata for the first few records
for i in range(len(query_results["ids"])):
    print(f"Record {i + 1}:")
    print(f"ID: {query_results['ids'][i]}")  # Print the ID of the record
    print(f"Metadata: {query_results['metadatas'][i]}")  # Print metadata
    print("=" * 50)


ValueError: At least one of one of embeddings, documents, images, uris must be provided in query.

In [18]:
# Query the first 5 records
query_results = multimodal_db.query(
    query_embeddings=None,  # We're not querying for specific embeddings here
    n_results=5,  # Number of results to retrieve
    where={"Breast_Composition": "Fatty"},  # Filter by metadata field
    include=["ids", "metadatas", "documents", "uris"]
)


ValueError: At least one of one of embeddings, documents, images, uris must be provided in query.

In [15]:
# Query the collection to retrieve records where the URI matches a specific value
query_results = multimodal_db.query(
    query_uris=["/mnt/data1/raiyan/breast_cancer/datasets/dmid/pixel_level_annotations/png_images/IMG002.png"],  # List of URIs to search for
    n_results=5,  # Specify the number of results to retrieve
    include=["metadatas", "documents", "uris"],  # Include metadata along with other fields
    where={"uris": "/mnt/data1/raiyan/breast_cancer/datasets/dmid/pixel_level_annotations/png_images/IMG002.png"}  # Filter by URI
)

# Print metadata for the filtered records
for i in range(len(query_results["ids"])):
    print(f"Record {i + 1}:")
    print(f"ID: {query_results['ids'][i]}")  # Print the ID of the record
    print(f"Metadata: {query_results['metadatas'][i]}")  # Print metadata
    print(f"URI: {query_results['uris'][i]}")  # Print URI
    print("=" * 50)


Record 1:
ID: []
Metadata: []
URI: []


In [21]:
import os
import json
from glob import glob
from PIL import Image
import torch
import open_clip
from torchvision import transforms
import chromadb

chroma_client = chromadb.PersistentClient(path="./chroma_db")  # Persistent storage
multimodal_db = chroma_client.get_or_create_collection(name="image_embeddings")


# Load OpenCLIP model
device = "cuda" if torch.cuda.is_available() else "cpu"
model = open_clip.create_model("ViT-B/32", pretrained="laion2b_s34b_b79k").to(device)

# ✅ Remove tokenizer (not needed for vision-only models)

# Preprocessing for OpenCLIP (manual transform)
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to model input size
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.481, 0.457, 0.408), std=(0.268, 0.261, 0.275))  # CLIP normalization
])





# Function to generate image embeddings
def get_image_embedding(image):
    image = preprocess(image).unsqueeze(0).to(device)
    with torch.no_grad():
        embedding = model.encode_image(image)
    return embedding.cpu().numpy().flatten().tolist()  # Convert to list of floats

# Define directories
image_directory = '/mnt/data1/raiyan/breast_cancer/datasets/dmid/pixel_level_annotations/png_images/'
json_directory = '/mnt/data1/raiyan/breast_cancer/VLMs-for-Mammograms/GROUND-TRUTH-REPORTS'

# Initialize lists
ids, uris, metadatas, image_embeddings = [], [], [], []

# Process images
processed_count = 0  
for img_path in glob(os.path.join(image_directory, '*.png')):  
    img_name = os.path.basename(img_path)  
    json_path = os.path.join(json_directory, img_name.replace('.png', '.json'))  

    if os.path.exists(json_path):
        with open(json_path, 'r') as json_file:
            metadata = json.load(json_file)

        metadata_entry = {
            'Breast_Composition': metadata.get('Breast_Composition', 'N/A'),
            'BIRADS': metadata.get('BIRADS', 'N/A'),
            'Findings': metadata.get('Findings', 'N/A')
        }

        # Load and process image
        image = Image.open(img_path).convert("RGB")
        embedding = get_image_embedding(image)

        # Store details
        ids.append(img_name)
        uris.append(img_path)
        metadatas.append(metadata_entry)
        image_embeddings.append(embedding)

        processed_count += 1
        print(f"Indexed {processed_count}: {img_name}")
        
# Inspect the data
for idx, img_path in enumerate(uris):
    print(f"URI {idx}: {img_path}")
    
for idx, embedding in enumerate(image_embeddings):
    print(f"Embedding {idx}: {embedding[:5]}")  # Print first 5 values of each embedding


# Store in ChromaDB
multimodal_db.update(
    ids=ids,
    uris=uris,
    embeddings=image_embeddings,  # ChromaDB requires embeddings
    metadatas=metadatas
)


print(f"Total images indexed: {processed_count}")


Indexed 1: IMG315.png
Indexed 2: IMG156.png
Indexed 3: IMG282.png
Indexed 4: IMG062.png
Indexed 5: IMG145.png
Indexed 6: IMG103.png
Indexed 7: IMG458.png
Indexed 8: IMG414.png
Indexed 9: IMG107.png
Indexed 10: IMG029.png
Indexed 11: IMG148.png
Indexed 12: IMG319.png
Indexed 13: IMG353.png
Indexed 14: IMG407.png
Indexed 15: IMG209.png
Indexed 16: IMG018.png
Indexed 17: IMG422.png
Indexed 18: IMG510.png
Indexed 19: IMG496.png
Indexed 20: IMG379.png
Indexed 21: IMG099.png
Indexed 22: IMG453.png
Indexed 23: IMG069.png
Indexed 24: IMG033.png
Indexed 25: IMG278.png
Indexed 26: IMG158.png
Indexed 27: IMG266.png
Indexed 28: IMG012.png
Indexed 29: IMG451.png
Indexed 30: IMG084.png
Indexed 31: IMG081.png
Indexed 32: IMG286.png
Indexed 33: IMG179.png
Indexed 34: IMG086.png
Indexed 35: IMG481.png
Indexed 36: IMG336.png
Indexed 37: IMG295.png
Indexed 38: IMG074.png
Indexed 39: IMG174.png
Indexed 40: IMG172.png
Indexed 41: IMG380.png
Indexed 42: IMG335.png
Indexed 43: IMG415.png
Indexed 44: IMG433.p

In [35]:
# Function to retrieve top 3 similar images from ChromaDB
def retrieve_similar_images(query_image_path):
    # Load and preprocess the query image
    query_image = Image.open(query_image_path).convert("RGB")
    query_embedding = get_image_embedding(query_image)
    
    # Query ChromaDB for the most similar embeddings
    results = multimodal_db.query(
        query_embeddings=[query_embedding],  # Query using the passed image embedding
        n_results=3  # Retrieve top 3 results
    )
    
    # Inspect and log the full results to understand their structure
    print("Full Query Results:")
    print(results)
    
    print("ID: ", results['ids'])
    print("Documents: ", results['documents'])
    print("Metadatas: ", results['metadatas'])
    # print("URIS: ", results['uris'])
    # print("Embeddings: ", results['embeddings'])


    
    # Display the top 3 similar images and their metadata
    for idx, result in enumerate(results['documents']):
        # Extract metadata
        metadata = results['metadatas'][idx]
        
        # Print the image and metadata
        print(f"Similar Image {idx+1}:")
        print(f"  Image Path: {result}")  # The path of the similar image
        
        # Ensure metadata is a dictionary and access its attributes
        if isinstance(metadata, dict):
            print(f"  Breast Composition: {metadata.get('Breast_Composition', 'N/A')}")
            print(f"  BIRADS: {metadata.get('BIRADS', 'N/A')}")
            print(f"  Findings: {metadata.get('Findings', 'N/A')}")
        else:
            print("  Metadata format error, expected a dictionary.")
        
        print("-" * 40)

# Example usage
query_image_path = "/mnt/data1/raiyan/breast_cancer/datasets/dmid/pixel_level_annotations/png_images/IMG001.png"  # Path to the image you want to query
retrieve_similar_images(query_image_path)


Full Query Results:
{'ids': [['IMG001.png', 'IMG403.png', 'IMG094.png']], 'embeddings': None, 'documents': [[None, None, None]], 'uris': None, 'data': None, 'metadatas': [[{'BIRADS': 3, 'Breast_Composition': 'predominantly fibro fatty breast parenchyma (ACR B)', 'Findings': 'irregular ill-defined soft opacity with microcalcifications suggests malignant lesion (BIRADS-5)\nsmall well defined soft nodular opacity -- benign lesion (BIRADS-3)\nbenign and vascular calcifications\nSkin and nipple - no abnormality\nNo axillary adenopathy'}, {'BIRADS': 3, 'Breast_Composition': 'fibro fatty and glandular breast parenchyma (ACR B)', 'Findings': 'an ill-defined soft opacity seen in the upper quadrant of the breast suggest probably a benign lesion \n(BIRADS 3)\n\nbenign-looking calcifications seen in the breast. \n\nskin, nipple, and pectoral muscle appear normal. \n\nfew small benign-looking axillary nodes seen. \n\nBIRADS:3'}, {'BIRADS': 1, 'Breast_Composition': 'dense glandular breast parenchyma

In [39]:


# Extracting the IDs and metadata and storing them in the required format.
formatted_data = []

for img_id, metadata in zip(results['ids'][0], results['metadatas'][0]):
    formatted_entry = {
        "IMG-ID": img_id,
        "BREAST-COMPOSITION": metadata['Breast_Composition'].replace('\n', ' ').strip(),
        "BIRADS": str(metadata['BIRADS']),
        "FINDINGS": metadata['Findings'].replace('\n', ' ').strip()
    }
    formatted_data.append(formatted_entry)

# Storing the results in separate variables
entry_1 = f"""{{
    "IMG-ID": "{formatted_data[0]['IMG-ID']}",
    "BREAST-COMPOSITION": "{formatted_data[0]['BREAST-COMPOSITION']}",
    "BIRADS": "{formatted_data[0]['BIRADS']}",
    "FINDINGS": "{formatted_data[0]['FINDINGS']}"
}}"""

entry_2 = f"""{{
    "IMG-ID": "{formatted_data[1]['IMG-ID']}",
    "BREAST-COMPOSITION": "{formatted_data[1]['BREAST-COMPOSITION']}",
    "BIRADS": "{formatted_data[1]['BIRADS']}",
    "FINDINGS": "{formatted_data[1]['FINDINGS']}"
}}"""

entry_3 = f"""{{
    "IMG-ID": "{formatted_data[2]['IMG-ID']}",
    "BREAST-COMPOSITION": "{formatted_data[2]['BREAST-COMPOSITION']}",
    "BIRADS": "{formatted_data[2]['BIRADS']}",
    "FINDINGS": "{formatted_data[2]['FINDINGS']}"
}}"""

# Now the formatted strings are stored in entry_1, entry_2, and entry_3
print(entry_3)



{
    "IMG-ID": "IMG094.png",
    "BREAST-COMPOSITION": "dense glandular breast parenchyma, which lowers the sensitivity of mammograms and may obscure small lesions (ACR D).",
    "BIRADS": "1",
    "FINDINGS": "benign-looking coarse calcification in the lower quadrant.   skin, nipple, and pectoral muscle appear normal. benign-looking axillary adenopathy."
}


AttributeError: 'list' object has no attribute 'length'

In [14]:
# Example usage
query_image_path = "/mnt/data1/raiyan/breast_cancer/datasets/dmid/pixel_level_annotations/png_images/IMG002.png"  # Path to the image you want to query
retrieve_similar_images(query_image_path)

Similar Image 1:
  Image Path: [None, None, None]
  Metadata format error, expected a dictionary.
----------------------------------------


In [25]:
# Example check: retrieve a few entries from ChromaDB to inspect
print(multimodal_db.get(ids=['IMG001.png']))


{'ids': ['IMG001.png'], 'embeddings': None, 'documents': [None], 'uris': None, 'data': None, 'metadatas': [{'BIRADS': 3, 'Breast_Composition': 'predominantly fibro fatty breast parenchyma (ACR B)', 'Findings': 'irregular ill-defined soft opacity with microcalcifications suggests malignant lesion (BIRADS-5)\nsmall well defined soft nodular opacity -- benign lesion (BIRADS-3)\nbenign and vascular calcifications\nSkin and nipple - no abnormality\nNo axillary adenopathy'}], 'included': [<IncludeEnum.documents: 'documents'>, <IncludeEnum.metadatas: 'metadatas'>]}
